In [ ]:
# =========================
# IMPORT LIBARY ===========
# =========================

import pandas as pd # Import library untuk manipulating dataset
import seaborn as sns # Import library untuk visualisasi EDA
import numpy as np # Import library untuk manipulating number data

from pandas.api.types import is_string_dtype, is_numeric_dtype # Import library untuk membaca type data column Dataframe
from matplotlib import pyplot as plt # Import library untuk Exploratory Data Analysis
from tkinter import * # Import library untuk pembuatan GUI
from tkinter import ttk # Import library turunan TKInter untuk tambahan widget dan component

data_path = 'datasets/diabetes_data_upload.csv' # membuat variable untuk menyimpan path data ke dataset

df = pd.read_csv(data_path) #membaca dataset dari variable data_path 

df.head() # menampilkan 5 data teratas

In [ ]:
df.shape # cek dimensi data sebelum masuk pre-processing

In [ ]:
df.nunique() # mengecek data unik pada setiap kolom

In [ ]:
df.info() # menampilkan informasi dari data frame

In [ ]:
# barplot melihat jumlah data duplicate
plt.figure(figsize=(6, 8))
plt.title("Diagram presentase jumlah data duplikat")

count_duplicate = df.duplicated().sum()
non_duplicate = len(df) - count_duplicate
total = count_duplicate + non_duplicate

print("\nIni Data Duplikat :", count_duplicate)
print("\nIni Data Tidak Duplikat : ",non_duplicate)
print("\nTotal Data :", total)

precentages = [
    count_duplicate / total * 100,
    non_duplicate / total * 100
    
]

res = sns.barplot(
    x=['Duplikat', 'Tidak Duplikat'],
    y=[count_duplicate, non_duplicate]
)

for i, p in enumerate(res.patches):
    res.text(
        p.get_x() + p.get_width() / 2,
        p.get_height(),
        f'{precentages[i]:.1f}%',
        ha='center',
        va='bottom'
    )

plt.ylabel("Jumlah Data")
plt.show()


In [ ]:
# =====================
# DATA PREPROCESSING ==
# =====================

df.isna().sum() # mengecek data yang kosong
df.duplicated().sum() # mengecek data yang duplikat

In [ ]:
df = df.drop_duplicates() # menghapus data duplikat dan melakukan reset index
df = df.fillna(df.mode().iloc[0])

In [ ]:
df.shape #mengecek dimensi data setelah menghapus data duplikat

In [ ]:
df.nunique()

In [ ]:
def encode_column(col): # membuat function untuk melakukan encoding data dengan variable unique
    unique_vals = col.unique()
    mapping = {val: idx for idx, val in enumerate(unique_vals)}
    return col.map(mapping)


for col in df.columns: # melakukan encoding data pada column variable df
    if df[col].dtype == "object":
        df[col] = encode_column(df[col])
        
df.columns = df.columns.str.lower().str.replace(' ', '_')
        
df.head(10) # mengecek 10 data teratas yang sudah di encode